# Pandas
A high level Python package to work with structured data.
Tools are built on top of NumPy.

The name comes from PANel DAta.

## Series
* One dimensional array
* Can be any data type, also Python dict or ndarray from numpy 
* Series can have name

## Data frame
* Two dimensional
* Table like object
* Takes in lists, dicts, ndarrays, Series and so on

In [1]:
import pandas as pd
# Pandas requires numpy installation but not import
import numpy as np

## Series

In [13]:
#Create series
group = pd.Series(["A","B","C","A","B","B","A"])
income = pd.Series([5000,2000,4000,3000,1500,1000,4500])

#Series to data frame
df = pd.DataFrame({'group':group, 'income':income})

df.head(10)

,group,income
0,A,5000
1,B,2000
2,C,4000
3,A,3000
4,B,1500
5,B,1000
6,A,4500


## Generate data

In [3]:
#Random integers
df_gen = pd.DataFrame(np.random.randint(0,100,size=(100, 2)), columns=['auto_1','auto_2'])
df_gen.head()

,auto_1,auto_2
0,89,57
1,68,45
2,55,73
3,98,60
4,24,69


In [4]:
#Dates
date_list = pd.date_range(start='1/1/2018', end='1/08/2018')
print(date_list)

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08'],
              dtype='datetime64[ns]', freq='D')


## Concatenate columns

In [5]:
#Concatenate rank to original data frame
df_concat = pd.concat([df, df_gen], axis=1)
df_concat.head()

,group,income,auto_1,auto_2
0,A,5000.0,89,57
1,B,2000.0,68,45
2,C,4000.0,55,73
3,A,3000.0,98,60
4,B,1500.0,24,69


## Filter rows

In [6]:
#df.loc[condition, columns]
bool_1 = df['income'] >= 3000
bool_2 = df['income'] <= 4500
df.loc[bool_1 & bool_2, ['income']]

,income
2,4000
3,3000
6,4500


## Group by

In [7]:
#Sum by group
df_sum = df.groupby(['group']).sum()
df_sum.head()

,income
group,
A,12500
B,4500
C,4000


In [8]:
#Rank by group
df_rank = df.groupby("group")["income"].rank(method="dense")
df_rank.head()

0    3.0
1    3.0
2    1.0
3    1.0
4    2.0
Name: income, dtype: float64

In [9]:
#Two groups
df_new_group = df.copy()
df_new_group['group_2'] = ["A1","B1","C1","A1","B1","B2","A2"]
df_new_group.groupby(['group', 'group_2'])['income'].sum()

group  group_2
A      A1         8000
       A2         4500
B      B1         3500
       B2         1000
C      C1         4000
Name: income, dtype: int64

In [10]:
#Multiple aggregations
#Note: agg function will be deprecated in th future. Use apply instead.
def my_agg(x):
    names = {
        'group_count': x['group'].count(),
        'income_sum': x['income'].sum(),
        'income_mean': x['income'].sum()/x['income'].count(),
        'income_mean_plus_rand_100': x['income'].mean() + np.random.rand()*100,
    }
    return pd.Series(names)
    
df_new_group.groupby(['group', 'group_2']).apply(my_agg)

group_count  income_sum  income_mean  income_mean_plus_rand_100
group group_2                                                                 
A     A1               2.0      8000.0       4000.0                4030.117239
      A2               1.0      4500.0       4500.0                4549.056066
B     B1               2.0      3500.0       1750.0                1790.956708
      B2               1.0      1000.0       1000.0                1006.013489
C     C1               1.0      4000.0       4000.0                4031.290678

## Rolling

In [15]:
df_roll = df.copy()
df_roll = df_roll["income"].rolling(center=False, window=3).sum()
df_roll.head()

0        NaN
1        NaN
2    11000.0
3     9000.0
4     8500.0
Name: income, dtype: float64

## Speed test

In [ ]:
import datetime as dt

#Get start time
t_start = dt.datetime.now()

#Sort data frame size of million by two conditions
st_rows = 10*1000000 #millions
df_st = pd.DataFrame(np.random.randint(0,1000,size=(st_rows, 4)), columns=list('ABCD'))
df_st = df_st.sort_values(by=['A', 'B'])

#Get end time
t_end = dt.datetime.now()

#Print results
print(df_st.head())
print("\nDuration")
print(t_end-t_start)

## Apply

In [ ]:
df_apply = df.copy()
df_apply['add_random'] = df['income'].apply(lambda x: np.random.rand()*500 + x)
df_apply

## Selecting subsets from dataframe

In [31]:
#Set ne index to show examples
df_sel = df.copy()
df_sel.index = [2,4,6,8,10,12,14]

In [32]:
#Selecting single column by brackets. Use list instead of string to select multiple.
print(df_sel["group"])

2     A
4     B
6     C
8     A
10    B
12    B
14    A
Name: group, dtype: object


In [37]:
#Selecting rows and columns based on integer positions
print(df_sel.iloc[0:5,0:2])

   group  income
2      A    5000
4      B    2000
6      C    4000
8      A    3000
10     B    1500


In [36]:
#Selecting rows by labels in index and columns by labels on column names.
print(df_sel.loc[0:5, ["group","income"]])

  group  income
2     A    5000
4     B    2000


In [35]:
#Selecting rows by boolean array with .loc
is_big = df_sel["income"] > 3500
print(df_sel.loc[is_big, ["group","income"]])

   group  income
2      A    5000
6      C    4000
14     A    4500
